In [31]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [32]:
import logging
import pandas as pd

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [36]:
customer = "Asambeauty"
audience_id = "6687daf21767991e3bdfb414"
session_date = "2025-06-25"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [37]:
audiences = call_api_with_accountId(
    f"{url}/audiences/query",
    account_id,
    {"id":audience_id},
    logging
)
if len(audiences) != 1:
    raise Exception(f"more or less than one source was found: {audiences}")
list_treatments = audiences[0]["config"]["treatments"]
print(f"treatments = {len(list_treatments)}")
list_treatments

In [38]:
treatments_response = call_api_with_accountId(
    f"{url}/treatments/query",
    account_id,
    {"id":list_treatments},
    logging
)
treatments_df = pd.json_normalize(treatments_response)
treatments_df

In [39]:
source = audiences[0]["source"]
source

In [40]:
sources = call_api_with_accountId(
    f"{url}/sources/query",
    account_id,
    {"_id":source},
    logging
)
sources

In [41]:
urlTrackingParam = sources[0]["options"]["urlTrackingParam"]
urlTrackingParam

In [42]:
sessions = send_to_innkeepr_api_paginated(
    f"{url}/sessions/query",
    account_id,
    {"created":{"$gte":session_date}},
    logging
)
sessions_df = pd.json_normalize(sessions)
sessions_df

In [ ]:
sessions_externalIds = sessions_df[f"campaign.{urlTrackingParam}"].dropna().unique().tolist()
sessions_externalIds

In [ ]:
treatments_in_sessinos = treatments_df[treatments_df["externalId"].isin(sessions_externalIds)]
treatments_in_sessinos

In [ ]:
treatments_df[treatments_df["id"]=="684b6dc7d3c557961f4001d5"]